In [1]:
import FinanceDataReader as fdr
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import yfinance as yf

# 분석 시작일

date_to = datetime.now().date() - timedelta(days=3)

# 과거 데이터 확인 기간
past_data_years = 1
delta = relativedelta(years=past_data_years)

# 데이터 시작일
date_from = date_to - delta
str_date_from = date_from.strftime('%Y-%m-%d')
str_date_to = date_to.strftime('%Y-%m-%d')
print('데이터 시작일 : ', str_date_from, str_date_to)

# 시장 종류
# 'S&P500', 'NASDAQ', 'NYSE', 'AMEX', 'SSE', 'SZSE', 'HKEX', 'TSE', 'HOSE', 'KRX'
market = 'NASDAQ'

데이터 시작일 :  2020-11-09 2021-11-09


In [16]:
# 시장별 ticker 가져오기
market_list = ['S&P500', 'NASDAQ', 'NYSE', 'KRX']

ticker_dict = dict()
market_df = pd.DataFrame()

for i in range(len(market_list)):
    
    market = market_list[i]
    df = pd.DataFrame()
    if i == 0:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = df
        print(market_list[i])
    else:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = market_df.append(df)
        print(market_list[i])
market_df.to_pickle()

C:\Program Files\Python38\lib\site-packages\FinanceDataReader\wikipedia\listing.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('\.', '')
C:\Users\is\AppData\Local\Temp/ipykernel_35360/1687164111.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('/.', '')


S&P500


100%|██████████| 4304/4304 [00:05<00:00, 753.07it/s]
C:\Users\is\AppData\Local\Temp/ipykernel_35360/1687164111.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('/.', '')


NASDAQ


100%|██████████| 3400/3400 [00:04<00:00, 798.18it/s]


NYSE
KRX


,Symbol,Name,Sector,Industry,Market,IndustryCode,ListingDate,SettleMonth,Representative,HomePage,Region
0,MMM,3M,Industrials,Industrial Conglomerates,S&P500,NaN,NaT,NaN,NaN,NaN,NaN
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,S&P500,NaN,NaT,NaN,NaN,NaN,NaN
2,ABBV,AbbVie,Health Care,Pharmaceuticals,S&P500,NaN,NaT,NaN,NaN,NaN,NaN
3,ABMD,Abiomed,Health Care,Health Care Equipment,S&P500,NaN,NaT,NaN,NaN,NaN,NaN
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,S&P500,NaN,NaT,NaN,NaN,NaN,NaN


In [18]:
market_df.groupby('Market')['Symbol'].nunique()

Market
KRX       6504
NASDAQ    4304
NYSE      3400
S&P500     505
Name: Symbol, dtype: int64

In [ ]:
def make_inverstment_performance_indicator(df, target_price, date):
    daily_rtn = 'daily_rnt' # 일별 수익률
    st_rtn = 'st_rtn'  # 전략 수익률 수익률 누적의 곱

    df[daily_rtn] = df[target_price].pct_change()  # 일별 수익률
    df[st_rtn] = (1 + df[daily_rtn]).cumprod()  # 전략 수익률 = 수익률의 누적 곱
    
    # CAGR = Compound Annual Growth Rate 연평균 복리 수익률, 수익률이 높을 수록 좋다.
    CAGR = df.loc[date, st_rtn] ** (252./len(df.index)) -1  # 특정 시점까지의  최종 누적 수익률의 누적 연도 제곱근, 금융공학에서 1년은 252 영업일

    # MDD = Maximum Draw Down 최대 낙폭, MDD 값이 크면 변동성이 심한 주식
    historical_max = df[target_price].cummax()
    daily_drawdown = df[target_price] / historical_max - 1.0
    historical_dd = daily_drawdown.cummin()
    MDD = historical_dd.min()

    # VOL = Volatility 휘발성, 변동성
    VOL = np.std(df[daily_rtn]) * np.sqrt(252.)

    # Sharp = ex-post Sharp ratio, 사후적 사프 비율
    # 샤프지수와 같은 공식을 사용하지만, 자산의 실현 수익률을 사용한다는 점에서 다르다.
    # 샤프지수는 위험 대비 수익성 지표를 의미함
    # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
    # 보통 1 이상이면 좋다고 판단함

    Sharp = np.mean(df[daily_rtn]) / VOL

    return CAGR, Sharp, VOL, MDD
    


In [ ]:
data = yf.Ticker('MSFT')
df = data.history(period='1d', start=str_date_from, end=str_date_to)['Close'].reset_index().set_index('Date')
df.head()

In [ ]:
df.index[-1]

In [ ]:
select_stock_dict = dict()

CAGR_criteria = 0.05 # 연평균 복리 수익률(Compound Annual Growth Rate)
Sharp_criteria = 0.01  # 샤프 지수는 0.01 이상 되면 좋다고 판단한다. (VOL을 곱하기 100 해주면 Sharp는 1)
VOL_criteria = 0.60  # 변동성 지수는 60 이상 -> 하루하루 변동성이 크면서 성장해야 단타를 자주 하지 않을까?
MDD_criteria = -0.90  # 최대 낙폭은 1년 이내에 50%까지 하락한 경우가 있으면 허용한다.
target_price = 'Close'  # 일별 종가를 기준으로 계산한다.

for ticker in ticker_dict[market_list[0]]:
    df = yf.Ticker(ticker).history(period='1y')
    # df = fdr.DataReader(ticker, str_date_from, str_date_to)[target_price].reset_index().set_index('Date')
    
    if date_to in pd.to_datetime(df.index.tolist()):
        CAGR, Sharp, VOL, MDD = make_inverstment_performance_indicator(df, target_price, str_date_to)
        # print(ticker, CAGR, Sharp, VOL, MDD)
        if CAGR >= CAGR_criteria and Sharp >= Sharp_criteria and VOL >= VOL_criteria and MDD >= MDD_criteria:
            if ticker not in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = None,
                    Sharp = None,
                    VOL = None,
                    MDD = None
                )
            if ticker in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = CAGR,
                    Sharp = Sharp,
                    VOL = VOL,
                    MDD = -1*MDD*100
                )
select_stock_dict

In [ ]:
fdr.DataReader('AAPL', '2020-01-01', '2020-01-03')

In [ ]:
import pandas as pd

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

df_listing = pd.read_html(url, header=0)[0]

cols_ren = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', 

                    '상장일':'ListingDate', '결산월':'SettleMonth',  '대표자명':'Representative', 

                    '홈페이지':'HomePage', '지역':'Region', }

df_listing = df_listing.rename(columns = cols_ren)

df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x: '{:06d}'.format(x))

df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

df_listing

In [ ]:
ms_df = fdr.DataReader(nasdaq_ticker_df.iloc[0]['Symbol'], str_start_date, str_today_date)
ms_df.head()

In [ ]:
price_df = ms_df['Close'].reset_index().set_index('Date')

In [ ]:
from_date = '2020-01-01'
to_date = '2021-01-01'
price_df.loc[from_date:to_date].plot(figsize=(16,9))

In [ ]:
# 일별 수익률
price_df['daily_rtn'] = price_df['Close'].pct_change()
price_df.head()

In [ ]:
# 바이앤 홀드 전략의 누적 곱 계산
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head()

In [ ]:
price_df['st_rtn'].plot(figsize=(16,9))

In [ ]:
base_date = '2020-04-01'
tmp_df = price_df.loc[base_date:,['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = tmp_df.index[-1]
print('누적 수익 : ', tmp_df.loc[last_date, 'st_rtn'])
tmp_df.plot(figsize=(16,9))

In [ ]:
### 투자 성과 분석 지표
# 1. 연평균 복리 수익률
# 기하 평균 N 개의 변수를 모두 곱한 값의 제곱근
# 가장 최근일의 최종 누적 수익률의 누적 연도 제곱근을 구한다. 금융공학에서 1년은 252 영업일로 계산
# ** 제곱
CAGR = price_df.loc[str_today_date, 'st_rtn'] ** (252/len(price_df.index)) - 1
CAGR

In [ ]:
# 2. 최대 낙폭(MDD Maximum Daily Drawdown)
historical_max = price_df['Close'].cummax()
daily_drawdown = price_df['Close'] / historical_max - 1.0
historical_dd = daily_drawdown.cummin()
historical_dd.plot()


In [ ]:
historical_dd[-1]

In [ ]:
# 3. 변동성
# 변동성은 금융 자산의 방향성에 대한 불확실성과 가격 등락에 대한 위험 예상 지표로 해석
# 수익률의 표준편차를 변동성으로 계산함
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
VOL

In [ ]:
# 4. 샤프 지수
# 샤프 지수는 위험 대비 수익성 지표라고 볼 수 있다.
# Ra 는 자산 수익률 Rb 는 무위험 수익률
# 사후적 샤프 비율 ex-post Sharpe ratio
# 공식은 같지만 자산의 실현 수익률을 사용한다는 점에서 샤프 지수와 다름
# 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
Sharpe = np.mean(price_df['daily_rtn']) / np.std(price_df['daily_rtn']) * np.sqrt(252.)
Sharpe

In [ ]:
import threading, time, signal
from datetime import timedelta

wait_time_seconds = 1

class ProgramKilled(Exception):
    pass

def foo():
    print(time.ctime())

def signal_handler(signum, frame):
    raise ProgramKilled

class Job(threading.Thread):
    def __init__(self, interval, execute, *args, **kwargs):
        threading.Thread.__init__(self)
        self.daemon = False
        self.stopped = threading.Event()
        self.interval = interval
        self.execute = execute
        self.args = args
        self.kwargs = kwargs
    
    def stop(self):
        self.stopped.set()
        self.join()

    def run(self):
        while not self.stopeed.wait(self.interval.total_seconds()):
            self.execute(*self.args, **self.kwargs)


if __name__ == "__main__":
    signal.signal(signal.SIGTERM, signal_handler)
    signal.signal(signal.SIGINT, signal_handler)
    job = Job(interval=timedelta(seconds=wait_time_seconds), execute=foo)
    job.start()

    while True:
        try:
            time.sleep(1)
        except ProgramKilled:
            print("Program killed: running cleanup code")
            job.stop()
            break

In [2]:
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
post_message(myToken,channel,"test_message")

<Response [200]>
